In [1]:
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import os
import numpy as np
import tensorflow as tf

In [2]:
def load_images_from_folder(folders):
    class_images = []
    for folder in folders:
        images = []
        for filename in os.listdir(folder):
            img = Image.open(os.path.join(folder,filename))
            if img is not None:
                img = img.resize((299, 299))
                img_flip = ImageOps.flip(img)
                all_img = [img, img_flip]
                for i in all_img:
                    for r in [0, 90, 180, 270]:
                        images.append(np.array(i.rotate(r)))
        class_images.append(images)
    
    return class_images.copy()

In [3]:
def defined_input(class_images):
    features = []
    for images in class_images:
        for img in images:
            features.append(img)
    features = np.array(features)
    features = tf.keras.applications.inception_v3.preprocess_input(features, data_format=None)
    return features

In [4]:
#เตรียม output
def defined_output(class_images):
    label = []
    for count, images in enumerate(class_images):
        for i in range(len(images)):
            classes = len(class_images)
            output = np.zeros((classes,), dtype=int)
            output[count] = 1
            label.append(output)
    return np.array(label)

In [5]:
#load test_data
foldersname = []
for i in range(14):
    foldersname.append('./images/Head_Mid_Tail/class' + str(i) + '/test/head') #change file path this.

class_images = load_images_from_folder(foldersname)

X_head_test = np.copy(defined_input(class_images))
y_head_test = np.copy(defined_output(class_images))

In [6]:
#load test_data
foldersname = []
for i in range(14):
    foldersname.append('./images/Head_Mid_Tail/class' + str(i) + '/test/mid') #path_to_folder, change file path this.

class_images = load_images_from_folder(foldersname)

X_mid_test = np.copy(defined_input(class_images))
y_mid_test = np.copy(defined_output(class_images))

In [7]:
#load test_data
foldersname = []
for i in range(14):
    foldersname.append('./images/Head_Mid_Tail/class' + str(i) + '/test/tail') #path_to_folder, change file path this.

class_images = load_images_from_folder(foldersname)

X_tail_test = np.copy(defined_input(class_images))
y_tail_test = np.copy(defined_output(class_images))

In [8]:
model_head = tf.keras.models.load_model('./model/head_mid_tail/head_v2.h5') #path_to_model, change file path this.
model_mid = tf.keras.models.load_model('./model/head_mid_tail/mid_v2.h5') #path_to_model, change file path this.
model_tail = tf.keras.models.load_model('./model/head_mid_tail/tail_v2.h5') #path_to_model, change file path this.

In [9]:
predict_prob_head = model_head.predict(X_head_test)
predict_prob_mid = model_head.predict(X_mid_test)
predict_prob_tail = model_head.predict(X_tail_test)

In [10]:
def get_rank(predict_prob):
    top_2_list = []
    for pred in predict_prob:
        pred_tmp = pred.copy()
        first = np.argmax(pred_tmp)
        pred_tmp[first] = -1
        second = np.argmax(pred_tmp)
        top_2_list.append([first, second])
    return top_2_list.copy()

In [11]:
top_rank_head = get_rank(predict_prob_head)
top_rank_mid = get_rank(predict_prob_mid)
top_rank_tail = get_rank(predict_prob_tail)

In [12]:
"""f1-score"""
weights =  [[2, 1, 1],
          [2, 1, 1],
          [1, 2, 1],
          [1, 1, 2],
          [1, 2, 1],
          [2, 2, 1],
          [2, 1, 1],
          [2, 1, 1],
          [1, 2, 1],
          [2, 1, 2],
          [2, 2, 1],
          [1, 1, 2],
          [1, 2, 1],
          [1, 2, 1]]
"""ผู้เชี่ยวชาญ"""
# weights =  [[2, 1, 1],
#           [2, 1, 1],
#           [1, 2, 1],
#           [1, 2, 1],
#           [1, 2, 1],
#           [1, 2, 1],
#           [1, 2, 1],
#           [2, 2, 1],
#           [2, 2, 1],
#           [1, 2, 1],
#           [1, 2, 1],
#           [2, 1, 2],
#           [2, 2, 1],
#           [1, 2, 1]]
weights = np.array(weights)

In [13]:
scores = []
for head, mid, tail, prob_head, prob_mid, prob_tail in zip(top_rank_head, top_rank_mid, top_rank_tail, predict_prob_head, predict_prob_mid, predict_prob_tail):
    score_dict = {}
    for index, i in enumerate([head[0], mid[0], tail[0]]):
        if i not in score_dict:
            score_dict[i] = weights[i][index]
        else:
            score_dict[i] = score_dict[i] + weights[i][index]
    for i in [head[1], mid[1], tail[1]]:
        if i in score_dict:
            score_dict[i] = score_dict[i] + 1
    scores.append(score_dict)

In [14]:
y_pred = []
for score in scores:
    max = -1
    for key in score:
        if score[key] > max:
            max = score[key]
            pred = key
    y_pred.append(pred)
    
y_true = []
for i in y_head_test:
    y_true.append(np.argmax(i))

In [15]:
from sklearn.metrics import classification_report

target_names = []
for i in range(14):
    target_names.append('class' + str(i))
report = classification_report(y_true, y_pred, target_names = target_names)
print(report)

              precision    recall  f1-score   support

      class0       0.61      0.50      0.55        88
      class1       0.45      0.81      0.58        48
      class2       1.00      0.27      0.43        48
      class3       0.85      0.61      0.71        56
      class4       0.67      0.25      0.36         8
      class5       0.83      0.91      0.87        32
      class6       0.00      0.00      0.00        56
      class7       0.63      0.97      0.77        80
      class8       1.00      0.94      0.97        16
      class9       0.76      1.00      0.86        16
     class10       0.67      1.00      0.81        56
     class11       0.54      1.00      0.70        48
     class12       0.91      0.42      0.57        24
     class13       1.00      0.69      0.81        48

    accuracy                           0.67       624
   macro avg       0.71      0.67      0.64       624
weighted avg       0.67      0.67      0.62       624



C:\Users\acer\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
import pandas as pd 

#save report to excel
report_dict = classification_report(y_true, y_pred, target_names = target_names, output_dict=True)
df = pd.DataFrame(report_dict).transpose()
df.to_excel("body_f1-score.xlsx")